# Chapter 11: Training Deep Neural Networks

Di Bab 10, kita telah melatih jaringan saraf yang relatif dangkal. Namun, untuk masalah yang sangat kompleks, Anda mungkin perlu melatih Jaringan Saraf Dalam (Deep Neural Networks - DNNs) yang memiliki puluhan atau bahkan ratusan lapisan. Proses pelatihan ini tidaklah mudah dan seringkali dihadapkan pada beberapa masalah:

* **Vanishing/Exploding Gradients Problem:** Gradien menjadi sangat kecil atau sangat besar saat *backpropagation*, membuat lapisan bawah sulit untuk dilatih.
* **Kurangnya Data Pelatihan:** Jaringan besar memerlukan banyak data.
* **Pelatihan yang Lambat:** Jaringan besar bisa sangat lambat untuk dilatih.
* **Risiko Overfitting:** Model dengan jutaan parameter sangat berisiko *overfitting*.

Bab ini akan membahas setiap masalah ini dan menyajikan teknik-teknik untuk mengatasinya.

## The Vanishing/Exploding Gradients Problem

Masalah ini terjadi karena gradien seringkali menjadi semakin kecil (vanishing) atau semakin besar (exploding) saat algoritma bergerak mundur melalui lapisan-lapisan DNN. Beberapa solusi yang populer adalah:

### 1. Glorot and He Initialization
Inisialisasi bobot koneksi secara acak dengan cara yang benar sangat penting. Strategi inisialisasi Glorot (Xavier) dan He membantu menjaga varians sinyal tetap konstan saat bergerak maju dan mundur melalui jaringan, yang secara signifikan mengurangi risiko gradien yang menghilang atau meledak.

### 2. Nonsaturating Activation Functions
Fungsi aktivasi seperti Sigmoid dan Tanh cenderung "jenuh" di nilai ekstrim, membuat turunannya mendekati nol. Ini menyebabkan gradien berhenti mengalir. Solusinya adalah menggunakan fungsi aktivasi yang tidak jenuh, seperti **ReLU** dan variannya (**Leaky ReLU, ELU, SELU**).

### 3. Batch Normalization (BN)
Teknik ini menambahkan operasi di setiap lapisan yang me-nol-pusatkan dan menormalkan inputnya, lalu menskalakan dan menggesernya. Ini secara dramatis mengurangi masalah gradien yang tidak stabil dan memungkinkan penggunaan *learning rate* yang lebih tinggi, mempercepat konvergensi.

### 4. Gradient Clipping
Teknik lain untuk mengatasi gradien yang meledak adalah dengan "memotong" gradien selama *backpropagation* agar tidak pernah melebihi ambang batas tertentu.

In [ ]:
from tensorflow import keras

# Contoh penggunaan inisialisasi He dan Batch Normalization
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

## Reusing Pretrained Layers (Transfer Learning)

Melatih DNN yang sangat besar dari awal bukanlah ide yang baik. Sebaliknya, kita harus selalu mencoba mencari jaringan saraf yang sudah ada yang menyelesaikan tugas serupa, lalu menggunakan kembali lapisan bawahnya. Teknik ini disebut **Transfer Learning**.

Prosesnya melibatkan:
1. Memuat model yang sudah dilatih sebelumnya (misalnya, dilatih pada dataset ImageNet).
2. Membekukan bobot lapisan-lapisan bawahnya agar tidak berubah selama pelatihan awal.
3. Mengganti lapisan output dengan lapisan output baru yang sesuai dengan tugas kita.
4. Melatih model pada data kita. Lapisan baru akan belajar dengan cepat, sementara lapisan yang dibekukan tidak akan berubah.
5. (Opsional) Membuka beberapa lapisan beku dan melanjutkan pelatihan dengan *learning rate* yang lebih rendah untuk menyempurnakan model (*fine-tuning*).

In [ ]:
# Contoh Transfer Learning (konseptual)
# model_A = keras.models.load_model("my_model_A.h5")
# model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
# model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# Membekukan lapisan yang digunakan kembali
# for layer in model_B_on_A.layers[:-1]:
#     layer.trainable = False

# Melatih model (hanya lapisan baru yang akan dilatih)
# model_B_on_A.compile(...)
# history = model_B_on_A.fit(...)

## Faster Optimizers

Selain SGD biasa, ada beberapa *optimizer* yang lebih canggih yang dapat mempercepat pelatihan secara signifikan:

* **Momentum Optimization:** Menambahkan "momentum" pada pembaruan bobot, membantunya melewati *plateau* dan osilasi.
* **Nesterov Accelerated Gradient (NAG):** Varian dari momentum yang sedikit lebih cepat.
* **AdaGrad:** Memiliki *learning rate* yang adaptif, memberikan pembaruan yang lebih kecil untuk fitur yang sering muncul.
* **RMSProp:** Memperbaiki kelemahan AdaGrad yang cenderung berhenti terlalu cepat.
* **Adam & Nadam:** Menggabungkan ide dari momentum dan RMSProp. Adam adalah *optimizer* yang sangat populer dan sering menjadi pilihan *default* yang baik.

In [ ]:
# Menggunakan optimizer Adam di Keras
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
# model.compile(loss="...", optimizer=optimizer)

## Learning Rate Scheduling

Menemukan *learning rate* yang baik sangat penting. Alih-alih menggunakan *learning rate* konstan, kita bisa mendapatkan solusi yang lebih baik dan lebih cepat dengan mengubahnya selama pelatihan. Strategi ini disebut **learning rate scheduling**. Beberapa jadwal umum termasuk *power scheduling*, *exponential scheduling*, dan *piecewise constant scheduling*.

## Avoiding Overfitting Through Regularization

DNN yang besar sangat rentan terhadap *overfitting*. Selain *early stopping* dan *Batch Normalization*, ada teknik regularisasi lain yang populer:

* **L1 and L2 Regularization:** Menambahkan penalti pada *loss function* berdasarkan norma L1 atau L2 dari bobot koneksi. Ini mendorong bobot untuk tetap kecil.

* **Dropout:** Pada setiap langkah pelatihan, setiap neuron (kecuali di *output layer*) memiliki probabilitas `p` untuk "dijatuhkan" (diabaikan sementara). Ini memaksa neuron lain untuk belajar fitur yang lebih kuat dan tidak terlalu bergantung satu sama lain, menghasilkan jaringan yang lebih tangguh.

* **Max-Norm Regularization:** Membatasi norma L2 dari bobot koneksi yang masuk ke setiap neuron agar tidak melebihi nilai `r` tertentu.

In [ ]:
# Contoh model dengan Dropout
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])